In [5]:
import io 
import os
import sys
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import ollama

import gradio as gr


In [6]:
def Data():
    data=pd.read_csv("titanic_ dataset_final.csv")
    return data

df=Data()


    

In [11]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
df.shape

(891, 12)

In [9]:
df.size

10692

In [14]:
"""For Testing the peice of chunk for local system and used for the Gradio UI
    """

def generate_insights(df_summary, df_size, df_shape, df_info):
    #Generating output
    prompt = f"""
    **Dataset Analysis Report** 

    ###  **1. Dataset Summary**  
    ```
    {df_summary}
    ```

    ### **2. Dataset Size**  
    - **Total Elements:** {df_size}

    ### **3. Dataset Shape**  
    - **Rows:** {df_shape[0]}
    - **Columns:** {df_shape[1]}

    ### 🔹 **4. Dataset Info**  
    ```
    {df_info}
    ```

     **Based on this analysis, generate key insights about the dataset.**
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

#To_string conversion for LLM automatic Embeddings
buffer = io.StringIO()
df.info(buf=buffer)
df_info = buffer.getvalue()
df_summary = df.describe().round().to_string()
df_size = str(df.size)
df_shape = str(df.shape)

# Generate insights
insights = generate_insights(df_summary, df_size, df_shape, df_info)

print("Generated insights:\n\n", insights)


Generated insights:

 1. The Titanic dataset contains a total of 10,692 observations (rows). Out of these, only 891 instances are completely filled, indicating that a significant portion of the data is missing or unknown. This could be due to various reasons such as lack of documentation, loss of records during the journey, or privacy concerns.

2. The dataset has eight distinct features: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, and Embarked. However, only 891 instances contain data for all these features, while the rest have missing or unknown values.

3. The majority of passengers are male (645), with a smaller percentage of females (246). This suggests that the Titanic was predominantly a male-dominated voyage.

4. The average age of passengers is 30, with a standard deviation of 15. This indicates a wide range of ages, from infants to elderly passengers. However, it's important to note that over half (57%) of the passengers do not have an ex

In [11]:
"""Main method for the basic data analysis
"""
def eda_analysis(file):
    df=pd.read_csv(file)
    buffer = io.StringIO()
    df.info(buf=buffer)
    df_info = buffer.getvalue()

# Converting other dataset properties to strings
    df_summary = df.describe().round().to_string()
    df_size = str(df.size)
    df_shape = str(df.shape)
    insights=generate_insights(df_summary,df_info,df_shape,df_size)
    return insights

demo=gr.Interface(fn=eda_analysis,inputs="file",outputs="text")

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://45b1e377386a8cc77f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
